In [14]:
import pandas as pd
from PIL import Image, ImageOps
import os
import numpy as np
import torch
import shutil
from torchvision import transforms
import cv2
from albumentations.augmentations.transforms import Lambda, RandomBrightnessContrast
from albumentations.augmentations.geometric.transforms import ShiftScaleRotate, HorizontalFlip
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.augmentations.crops.transforms import RandomResizedCrop
from albumentations import Compose
import torch.nn as nn
import torch.utils.data as Data
from tqdm import tqdm
from torch.utils.data import WeightedRandomSampler
import mymodel
from d2l import torch as d2l

In [15]:
import collections
import math
import string
import csv

In [16]:
data_dir = './dataset/'

In [17]:
# data = pd.read_csv(os.path.join(data_dir, 'boneage-training-dataset.csv'))
# torch.tensor(data['id'].values)

In [18]:
def copyfile(fname, target_dir):
    """将文件复制到指定文件夹"""
    os.makedirs(target_dir, exist_ok=True)
    shutil.copy(fname, target_dir)


def read_labels(data_dir, fname):
    """读取标签，并将其标准化"""
    data = pd.read_csv(os.path.join(data_dir, fname))
    data = data.sort_values(by="id", ascending=True)
    data['male'] = data['male'].astype('float32')
    # data['boneage'] = data['boneage'].astype('float32')
    data = np.array(data)
    id = data[:, :1]
    age = data[:, 1:2]
    gender = data[:, 2:]
    return torch.tensor(id), torch.tensor(age), torch.tensor(gender)

# read_labels(data_dir, 'boneage-training-dataset.csv')


In [19]:
# d2l里的获取标签
def read_csv_labels_noMale(fname):
    """读取标签，返回字典格式`"""
    with open(fname, 'r') as f:
        # Skip the file header line (column name) 跳过文件头
        lines = f.readlines()[1:]
    tokens = [l.rstrip().split(',') for l in lines]
    # return dict(((id, boneage) for id, boneage, male in tokens))
    return dict((id, [boneage, male]) for id, boneage, male in tokens)

In [21]:
labels = read_csv_labels_noMale(os.path.join(data_dir, 'boneage-training-dataset.csv'))
labels['2277']
# labels = pd.read_csv(os.path.join(data_dir, 'boneage-training-dataset.csv'))
# row = labels.iloc[1]
# str(row['id'])
# filename = os.path.join(data_dir, 'boneage-valid-dataset', f"{row['id']}.png")
# filename
# type(row['boneage'])
# # len(labels)
# # for i in labels.values():
# #     print(i)
# # labels.get('1377', 0)
# collections.Counter(labels.values())
age_list = list(np.array(list(labels.values()))[:, 0])
class_count = collections.Counter(age_list)
class_count
# type(labels.iloc[0])

Counter({'180': 418,
         '12': 11,
         '94': 492,
         '120': 992,
         '82': 385,
         '138': 529,
         '150': 678,
         '156': 1113,
         '36': 106,
         '126': 198,
         '132': 1084,
         '57': 18,
         '188': 1,
         '4': 1,
         '24': 77,
         '60': 278,
         '159': 69,
         '106': 478,
         '30': 38,
         '149': 2,
         '33': 13,
         '78': 55,
         '88': 55,
         '162': 682,
         '32': 24,
         '54': 89,
         '174': 97,
         '144': 657,
         '27': 9,
         '170': 3,
         '113': 48,
         '108': 312,
         '136': 6,
         '165': 64,
         '42': 89,
         '21': 15,
         '69': 193,
         '50': 95,
         '90': 49,
         '192': 172,
         '84': 274,
         '96': 302,
         '216': 72,
         '140': 4,
         '100': 60,
         '72': 254,
         '168': 892,
         '48': 71,
         '139': 3,
         '186': 138,
         

In [9]:
def test(data_dir, labels, target_size):
    class_count = collections.Counter(labels.values())
    myBatch = {}
    lengthOfDataset = len(labels)
    for key in class_count.keys():
        myBatch[key] = math.ceil(target_size * class_count[key] / lengthOfDataset)
    label_count = {}
    for train_file in os.listdir(os.path.join(data_dir, 'boneage-training-dataset')):
        # 获取该文件的标签
        label = labels[train_file.split('.')[0]]
        # 获取文件的路径
        fname = os.path.join(data_dir, 'boneage-training-dataset', train_file)
        # 复制该文件到目标文件夹
        copyfile(fname, os.path.join(data_dir, 'train_valid_test', 
                                    'train_valid', label))
        if label not in label_count or label_count[label] < myBatch[label]:
            copyfile(fname, os.path.join(data_dir, 'train_valid_test', 
                                        'valid', label))
            # 将该标签加入记录， 若count中没有则默认返回默认值0
            label_count[label] = label_count.get(label, 0) + 1
        else:
            copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                        'train', label))

首先要获得每个年龄的数据集在总数据集中的占比

In [10]:
# new_labels = np.array(labels)
# type(n['180'])
# class_count = collections.Counter(labels.values())
# myBatch = {}
# lengthOfDataset = len(labels)
# target_size = 2000
# count = 0
# for key in class_count.keys():
#     myBatch[key] = math.ceil(target_size * class_count[key] / lengthOfDataset)
#     count += myBatch[key]
# myBatch
# print(new_labels)
# data = np.array(data)
# age = data[:, 1:2]
# age = torch.tensor(age)
# for i in range(age.shape[0]):
#     record[age[i]] += 1
# age

In [11]:
def creatSmallDataset(data_dir, labels, target_size, DF):
    """创建新的数据集"""
    train_list = [['id', 'boneage', 'male']]
    age_list = list(np.array(list(labels.values()))[:, 0])
    class_count = collections.Counter(age_list)
    myBatch = {}
    lengthOfDataset = len(labels)
    for key in class_count.keys():
        myBatch[key] = math.ceil(target_size * class_count[key] / lengthOfDataset)
    label_count = {}
    for idx, row in DF.iterrows():
        # 获取当前文件路径
        filename = os.path.join(data_dir, 'boneage-training-dataset', f"{row['id']}.png")
        # 获取该文件的标签
        label = str(row['boneage'])
        if label not in label_count or label_count[label] < myBatch[label]:
            copyfile(filename, os.path.join(data_dir, 'small-dataset', label))
            train_list.append([row['id'], row['boneage'], row['male']])
            # 将该标签加入记录， 若count中没有则默认返回默认值0
            label_count[label] = label_count.get(label, 0) + 1
    with open('./dataset/small-dataset.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for row in train_list:
            writer.writerow(row)
    # for train_file in os.listdir(os.path.join(data_dir, 'boneage-training-dataset')):
    #     # 获取改文件的id,type:str
    #     fID = train_file.split('.')[0]
    #     # 获取该文件的标签
    #     label = labels[fID]
    #     # 获取文件的路径
    #     fname = os.path.join(data_dir, 'boneage-training-dataset', train_file)
    #     # 复制该文件到目标文件夹
    #     if label not in label_count or label_count[label] < myBatch[label]:
    #         copyfile(fname, os.path.join(data_dir, 'boneage-small-dataset'))
    #         # 获取该图片的所有label
    #         row = DF.iloc[]
    #         valid_list.append([DF])

    #         # 将该标签加入记录， 若count中没有则默认返回默认值0
    #         label_count[label] = label_count.get(label, 0) + 1

In [12]:
def reorg_data(data_dir, target_size):
    labels = read_csv_labels_noMale(os.path.join(data_dir, 'boneage-training-dataset.csv'))
    df = pd.read_csv(os.path.join(data_dir, 'boneage-training-dataset.csv'))
    creatSmallDataset(data_dir, labels, target_size, df)
    # d2l.reorg_test(data_dir)


# batch_size = 32
# valid_ratio = 0.1
target_size = 1000
# reorg_data(data_dir, target_size)

已获得原数据集分布的小数据集，接下来将图片数小于10的图片增广到10

In [13]:
def apply(input_root, aug, aug_size, label_list):
    """对文件夹下的所有图片进行增广"""
    labels = [] # str的列表
    for root, dirs, files in os.walk(input_root):
        dirs_size = len(dirs)
        files_size = len(files)
        if(dirs_size > 0):# 第一层，没有文件，只获取标签
            labels = dirs
        elif(files_size < aug_size):
            need_size = aug_size - files_size
            files = sorted(files)
            for i in need_size:
                idx = i%files_size
                fname = files[idx]
                label = label_list[fname.split('.')[0]]
                
                
                    
        # print(f'root:{root}, dirs:{dirs}, files:{files}')
    # Y = [aug(img) for _ in range(num_rows * num_cols)]
    # d2l.show_images(Y, num_rows, num_cols, scale=scale)
apply('./dataset/small-dataset/', 0, 0, 0)